In [1]:
'''
E91
'''

'\nE91\n'

In [2]:
from qiskit import BasicAer,execute,QuantumCircuit,QuantumRegister,ClassicalRegister
import matplotlib.pyplot as plt
from numpy.random import randint,random_integers
import numpy as np
import math
backend = BasicAer.get_backend('dm_simulator')

In [3]:
options_without_noise={
    'plot': False,
}
options_noise = { 
    'plot': False,
"rotation_error": {'rx':[1.0, 0.0], 'ry':[1.0, 0.0], 'rz':[1.0,0.0]},
"tsp_model_error": [1.0, 0.0],
"thermal_factor": 1.0,
"decoherence_factor": 1.0,
"depolarization_factor": 1.0,
"bell_depolarization_factor": 1.0,
"decay_factor": 1.0,
}

def density_check(arr):
    ket_plus=[[0.5+0.j, 0.5+0.j],[0.5+0.j, 0.5+0.j]]  #0
    ket_minus=[[ 0.5+0.j, -0.5+0.j],[-0.5+0.j, 0.5+0.j]]  #1
    ket_45=[[0.5+0.j, 0.35355339-0.35355339j],[0.35355339+0.35355339j,0.5+0.j]]  #0
    ket_225=[[ 0.5+0.j, -0.35355339+0.35355339j],[-0.35355339-0.35355339j, 0.5+0.j]]  #1
    ket_plus_i=[[5.000000e-01+0.j , 3.061617e-17-0.5j],[3.061617e-17+0.5j, 5.000000e-01+0.j ]] #0
    ket_minus_i =[[ 5.00000000e-01+0.j , -9.18485099e-17+0.5j],[-9.18485099e-17-0.5j,  5.00000000e-01+0.j ]] #1
    ket_135=[[ 0.5+0.j, -0.35355339-0.35355339j],[-0.35355339+0.35355339j,  0.5+0.j]]  #0
    ket_315=[[0.5+0.j, 0.35355339+0.35355339j],[0.35355339-0.35355339j, 0.5+0.j]]   #1
    
    if((arr == ket_plus).all()):
        return "ket_plus"
    elif((arr == ket_minus).all()):
        return "ket_minus"
    elif((arr == ket_45).all()):
        return "ket_45"
    elif((arr == ket_225).all()):
        return "ket_225"
    elif((arr == ket_plus_i).all()):
        return "ket_plus_i"
    elif((arr == ket_minus_i).all()):
        return "ket_minus_i"
    elif((arr == ket_135).all()):
        return "ket_135"
    else:
        return "ket_315"
        
def max_enatangled_gen(n):
    entangled_pairs=[]
    for i in range(n):
        qc=QuantumCircuit(2,2)
        qc.h(0)
        qc.cx(0,1) #Creating Pair |00> + |11> 
        entangled_pairs.append(qc)
    return entangled_pairs

## All the measurement are over the xy plane
def alice_measurement(alice_bases,pair,envs):
    alice_measured_data=[]
    for i in range(n):
        if(alice_bases[i]==0):
            
            pair[i].measure(0,0,basis='X')#0 degree 
        elif(alice_bases[i]==1):
            
            pair[i].measure(0,0, basis='N', add_param=np.array([0.707106,0.70710,0]))#45 degree  
        else:
            pair[i].measure(0,0, basis='Y')#90 
        
        if(envs=="noiseless"):
            job = execute(pair[i], backend, **options_without_noise)
        else:
            job = execute(pair[i], backend, **options_noise)
        result = job.result()
        density_matrix = result.results[0].data.densitymatrix
        dm_check = density_check(density_matrix)
        if ((dm_check == "ket_minus") or (dm_check =="ket_225") or (dm_check == "ket_minus_i") or (dm_check =="ket_315")) :
            measurement=1
        else:
            measurement=0
        alice_measured_data.append(measurement)
    return alice_measured_data

def bob_measurement(bob_bases,pair,envs):
    bob_measured_data=[]
    for i in range(n):
        if(bob_bases[i]==0):
            pair[i].measure(1,1, basis='N', add_param=np.array([0.707106,0.707106,0]))#45 degree   
        elif(bob_bases[i]==1):
            pair[i].measure(1,1, basis='Y')#90 
        else:
            pair[i].measure(1,1, basis='N', add_param=np.array([-0.707106,0.707106,0]))#135 degree 
        
        if(envs=="noiseless"):
            job = execute(pair[i], backend, **options_without_noise)
        else:
            job = execute(pair[i], backend, **options_noise)
        result = job.result()
        density_matrix = result.results[0].data.densitymatrix
        dm_check = density_check(density_matrix)
        if ((dm_check == "ket_minus") or (dm_check =="ket_225") or (dm_check =="ket_minus_i") or (dm_check =="ket_315")) :
            measurement=1
        else:
            measurement=0
        bob_measured_data.append(measurement)
    return bob_measured_data

def key_selection(alice_bases,bob_bases,measurement):
    shifted_key=[]  
    for i in range(n): 
        if((alice_bases[i]==1 and bob_bases[i]==0) or (alice_bases[i]==2 and bob_bases[i]==1)):
            shifted_key.append(bits[i])
    return shifted_key


In [4]:
n =12
pair = max_enatangled_gen(n)

In [5]:
alice_bases = randint(3,size=n)
bob_bases = randint(3,size=n)
envs = "noiseless"
alice_data = alice_measurement(alice_bases,pair,envs)
bob_data = bob_measurement(bob_bases,pair,envs)
alice_shifted_key = key_selection(alice_bases,bob_bases,alice_data)
bob_shifted_key = key_selection(alice_bases,bob_bases,bob_data)
print(alice_shifted_key)
print(bob_shifted_key)


ValueError: operands could not be broadcast together with shapes (4,4) (2,2) 

In [59]:
from qiskit.quantum_info import DensityMatrix, partial_trace
qc1 = QuantumCircuit(2,2)

qc1.h(0)
qc1.cx(0,1) # psi = |00> + |11> / root (2)
rho_AB = DensityMatrix(qc1)

print(rho_AB)

DensityMatrix([[0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j],
               [0.5+0.j, 0. +0.j, 0. +0.j, 0.5+0.j]],
              dims=(2, 2))


In [56]:
#Alice
qc1.measure(0,0, basis='N', add_param=np.array([0.70710689672121151610311545332544,0.70710678118654752440084436210485,6.123233995736766e-17]))
job1 = execute(qc1, backend, shots= 1024 ,**options_without_noise)
result1 = job1.result()
density_matrix1 = result1.results[0].data.densitymatrix
print(density_matrix1)



[[0.25+0.j 0.  +0.j 0.  +0.j 0.25+0.j]
 [0.  +0.j 0.25+0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.25+0.j 0.  +0.j]
 [0.25+0.j 0.  +0.j 0.  +0.j 0.25+0.j]]


In [58]:

qc1.measure(0,0, basis='N', add_param=np.array([0.70710689672121151610311545332544,0.70710678118654752440084436210485,6.123233995736766e-17]))
job2 = execute(qc1, backend,shots = 1, **options_without_noise)
result2 = job2.result()

density_matrix2 = result2.results[0].data.densitymatrix
print(density_matrix2)



[[0.25+0.j 0.  +0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.25+0.j 0.  +0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.25+0.j 0.  +0.j]
 [0.  +0.j 0.  +0.j 0.  +0.j 0.25+0.j]]
